
# Missouri VA Access Analytics — Exploratory Data Analysis (EDA)

This notebook performs exploratory data analysis (EDA) on the **Missouri subset** of the Griffith (2024) dataset:  
*Appointment Wait Times for Primary and Specialty Care in Veterans Health Administration Facilities vs. Community Medical Centers*.

**Repository paths assumed:**  
- Cleaned data: `data/cleaned/cleaned_mo_waits.csv.gz`  
- Figures output: `figures/eda/`  
- Metadata/logs: `data/metadata/`

> Tip: Run this notebook from the project root so relative paths resolve correctly.


## 1) Environment Setup and Imports

In [ ]:

# Standard libraries
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Data & numeric
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
# (Optional) seaborn for quick EDA; comment out if you prefer pure matplotlib
try:
    import seaborn as sns
    sns.set_theme(context="notebook", style="whitegrid")
except Exception as e:
    print("[WARN] seaborn not available; proceeding with matplotlib only.")

# Missing data viz (optional)
try:
    import missingno as msno
except Exception as e:
    print("[WARN] missingno not installed. Install with `pip install missingno` if desired.")

# Config
np.random.seed(42)
plt.rcParams.update({
    "figure.figsize": (9, 5),
    "axes.titlesize": 13,
    "axes.labelsize": 11
})

# Paths
PROJECT_ROOT = Path.cwd()
DATA_CLEANED = PROJECT_ROOT / "data" / "cleaned"
FIG_DIR = PROJECT_ROOT / "figures" / "eda"
META_DIR = PROJECT_ROOT / "data" / "metadata"
FIG_DIR.mkdir(parents=True, exist_ok=True)
META_DIR.mkdir(parents=True, exist_ok=True)

DATA_FILE = DATA_CLEANED / "cleaned_mo_waits.csv.gz"
print(f"[INFO] Project root: {PROJECT_ROOT}")
print(f"[INFO] Using data file: {DATA_FILE}")
print(f"[INFO] Figures -> {FIG_DIR}")


## 2) Data Loading and Basic Inspection

In [ ]:

assert DATA_FILE.exists(), f"Cleaned data not found at {DATA_FILE}. Run the cleaning script first."
df = pd.read_csv(DATA_FILE, low_memory=False, parse_dates=["activitydatetime"], dtype_backend="numpy_nullable")

print("[INFO] Shape:", df.shape)
print("[INFO] Columns:", list(df.columns))

display(df.head(3))
display(df.tail(3))

# Basic info & summary
buf = []
df.info(buf=buf)
print("\n".join(map(str, buf)))
display(df.describe(include="all", datetime_is_numeric=True).T)


## 3) Data Validation and Integrity Checks

In [ ]:

# Ensure key columns exist
required = [
    "activitydatetime", "year", "care_setting", "specialty_category",
    "wait_days", "met_access_standard"
]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Logical checks
neg_waits = (df["wait_days"].fillna(pd.NA) < 0).sum()
too_long_waits = (df["wait_days"] > 365).sum()
dups = df.duplicated().sum()

print(f"[CHECK] Negative waits: {neg_waits}")
print(f"[CHECK] >365-day waits: {too_long_waits}")
print(f"[CHECK] Duplicate rows: {dups}")

# Clip extreme waits for visualization (keep original for analysis)
df["wait_days_clipped"] = df["wait_days"].clip(lower=0, upper=365)


## 4) Missing Values and Null Patterns

In [ ]:

null_counts = df.isna().sum().sort_values(ascending=False)
print("[INFO] Null counts by column:\n", null_counts.head(20))

# Visualize if missingno is available
if 'msno' in globals():
    try:
        msno.matrix(df.sample(min(50000, len(df)), random_state=42), figsize=(9,4))
        plt.title("Missingness Matrix (sample)")
        plt.tight_layout()
        plt.savefig(FIG_DIR / "missing_matrix.png", dpi=150)
        plt.show()

        msno.heatmap(df, figsize=(8,4))
        plt.title("Missingness Heatmap")
        plt.tight_layout()
        plt.savefig(FIG_DIR / "missing_heatmap.png", dpi=150)
        plt.show()
    except Exception as e:
        print(f"[WARN] missingno visualization failed: {e}")


## 5) Descriptive Statistical Analysis

In [ ]:

numeric_cols = df.select_dtypes(include=["number", "floating", "integer"]).columns.tolist()
desc = df[numeric_cols].describe(percentiles=[.05, .25, .5, .75, .95]).T
desc["skew"] = df[numeric_cols].skew(numeric_only=True)
desc["kurtosis"] = df[numeric_cols].kurtosis(numeric_only=True)
display(desc)

desc.to_csv(META_DIR / "eda_numeric_summary.csv")
print(f"[INFO] Wrote numeric summary -> {META_DIR / 'eda_numeric_summary.csv'}")


## 6) Distribution and Outlier Visualization

In [ ]:

# Histogram / KDE for wait_days
plt.figure()
plt.hist(df["wait_days_clipped"].dropna(), bins=50)
plt.title("Distribution of Wait Days (clipped to 0-365)")
plt.xlabel("wait_days")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(FIG_DIR / "wait_days_hist.png", dpi=150)
plt.show()

# Boxplot by care_setting (matplotlib implementation)
groups = [g.dropna().values for _, g in df.groupby("care_setting")["wait_days_clipped"]]
labels = df["care_setting"].dropna().unique().tolist()

plt.figure()
plt.boxplot(groups, labels=labels, showfliers=False)
plt.title("Wait Days by Care Setting (no outliers)")
plt.xlabel("care_setting")
plt.ylabel("wait_days (clipped)")
plt.tight_layout()
plt.savefig(FIG_DIR / "wait_days_by_setting_box.png", dpi=150)
plt.show()


## 7) Bivariate and Comparative Analyses

In [ ]:

# Grouped stats by care_setting and specialty_category
group_stats = (
    df.groupby(["care_setting", "specialty_category"])
      ["wait_days"].agg(["count", "mean", "median", "std"])
      .sort_values(["care_setting", "mean"], ascending=[True, True])
)
display(group_stats.head(20))
group_stats.to_csv(META_DIR / "eda_group_stats_setting_specialty.csv")

# Compliance rates by setting
comp = df.pivot_table(index="care_setting", values="met_access_standard", aggfunc=["mean", "count"])
display(comp)
comp.to_csv(META_DIR / "eda_compliance_by_setting.csv")


## 8) Correlation and Feature Relationships

In [ ]:

num_for_corr = df.select_dtypes(include=["number"]).copy()
corr = num_for_corr.corr(numeric_only=True)

plt.figure(figsize=(8,6))
im = plt.imshow(corr, aspect='auto')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title("Correlation Matrix (numeric features)")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.tight_layout()
plt.savefig(FIG_DIR / "correlation_matrix.png", dpi=150)
plt.show()


## 9) Temporal Trends and Policy Milestones

In [ ]:

# Annual trend in wait_days by care_setting
annual = (
    df.groupby(["year", "care_setting"])
      ["wait_days"].median().reset_index()
)

for setting in annual["care_setting"].unique():
    sub = annual[annual["care_setting"] == setting]
    plt.figure()
    plt.plot(sub["year"], sub["wait_days"], marker="o")
    plt.title(f"Median Wait Days by Year — {setting}")
    plt.xlabel("year")
    plt.ylabel("median wait_days")
    # Policy markers
    for x, label in [(2014, "VACAA"), (2018, "MISSION Act")]:
        plt.axvline(x=x, linestyle="--")
        plt.text(x+0.1, plt.ylim()[1]*0.9, label)
    plt.tight_layout()
    fname = FIG_DIR / f"trend_median_wait_by_year_{setting}.png"
    plt.savefig(fname, dpi=150)
    plt.show()
    print(f"[INFO] Saved {fname}")


## 10) Summary Visualization Snapshot

In [ ]:

# Save a compact set of figures already created; list them out for the report
from pprint import pprint
generated = sorted([p.name for p in FIG_DIR.glob("*.png")])
print("[INFO] Generated figures:")
pprint(generated)
(pd.Series(generated)).to_csv(META_DIR / "figures_manifest.csv", index=False)
print(f"[INFO] Wrote figures manifest -> {META_DIR / 'figures_manifest.csv'}")



## 11) Documentation and Version Control

- Key numeric summaries are saved under `data/metadata/`.
- Figures are saved under `figures/eda/` and can be embedded in Overleaf.
- Commit this notebook and generated artifacts to Git for traceability.
- Consider parameterizing file paths and filters for reuse.
